In [113]:
import pandas as pd

stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')


stores_train_copy= stores_train.copy()
stores_test_copy= stores_test.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns=['sales_channel_name'], inplace=True)

plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')

df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, how='left', on='plaace_hierarchy_id')

busstops = pd.read_csv('data/busstops_norway.csv')
busstops_copy = busstops.copy()
busstops_copy['busstop_lat'] = busstops_copy['geometry'].apply(lambda x: float(x.split(' ')[1][:-1]))
busstops_copy['busstop_lon'] = busstops_copy['geometry'].apply(lambda x: float(x.split(' ')[0][6:]))

In [ ]:
print('hello world')

In [1]:
from haversine import haversine, Unit
import time
import numpy as np


def find_closest_busstop(lat, lon):
    distances = busstops_copy.apply(lambda x: haversine((lat, lon), (x['busstop_lat'], x['busstop_lon']), unit=Unit.METERS), axis=1)
    return distances.min()


stores_train_copy['closest_busstop'] = stores_train_copy.apply(lambda x: find_closest_busstop(x['lat'], x['lon']), axis=1)
stores_test_copy['closest_busstop'] = stores_test_copy.apply(lambda x: find_closest_busstop(x['lat'], x['lon']), axis=1)


stores_train_copy.to_csv('data/stores_train_with_busstops.csv', index=False)
stores_test_copy.to_csv('data/stores_test_with_busstops.csv', index=False)


In [ ]:
from haversine import haversine, Unit

def count_closest_stores(lat, lon, lv2, col, radius):
    distances = df[df[col] == lv2].apply(lambda x: haversine((lat, lon), (x['lat'], x['lon']), unit=Unit.METERS), axis=1)
    return len(distances[distances < radius] > 0)

radius = 500

col = 'lv2_desc'

df_train['num_closest_stores_lv2'] = df_train.apply(lambda x: count_closest_stores(x['lat'], x['lon'], x['lv2_desc'], col, 500), axis=1)
df_test['num_closest_stores_lv2'] = df_test.apply(lambda x: count_closest_stores(x['lat'], x['lon'], x['lv2_desc'], col, 500), axis=1)

df_train[['store_id','num_closest_stores_lv2_500m']].to_csv('stores_train_num_closest_stores_lv2_500m.csv', index=False)
df_test[['store_id','num_closest_stores_lv2_500m']].to_csv('stores_test_num_closest_stores_lv2_500m.csv', index=False)


In [ ]:
# X_total = pd.concat([X, y], axis=1)

# X_encoded_target = X_total.copy()

# all_means = {}
# for column in categorical_features_target:
   
#    column_means = X_encoded_target.groupby(column)[column,'revenue'].mean().to_dict()['revenue']
   
#    X_encoded_target[column] = X_encoded_target.groupby(column)['revenue'].transform('mean')
   
#    all_means[column] = column_means
   
    
# X_encoded_target.drop(columns=['revenue'], inplace=True)



In [ ]:
# # label
# label_encoder = LabelEncoder()

# X_encoded_label = X.copy()

# X_encoded_label[categorical_features] = X_encoded_label[categorical_features].apply(label_encoder.fit_transform)

# log all the columns that are int or float


# # target
X_total = pd.concat([X, y], axis=1)

X_encoded_target = X_total.copy()

# remove revenue outliers that are more than 3 std away from mean in X_encoded_target

X_encoded_target = X_encoded_target[np.abs(X_encoded_target.revenue - X_encoded_target.revenue.mean()) <= (3 * X_encoded_target.revenue.std())]

all_means = {}

for column in categorical_features:
   
   column_means = X_encoded_target.groupby(column)[column,'revenue'].mean().to_dict()['revenue']
   
   X_encoded_target[column] = X_encoded_target.groupby(column)['revenue'].transform('mean')
   
   all_means[column] = column_means

y_target = X_encoded_target.revenue
    
X_encoded_target.drop(columns=['revenue'], inplace=True)

In [ ]:
random_state = 12


def objective(trial):
    

    param = {
        'random_state': 48,
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        #'n_estimators': trial.suggest_int('n_estimators', 10, 3000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'rmse',

    }
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    kf_scores = np.empty(5)
    
    model = xgb.XGBRegressor(**param)
    
    X_selected = X_encoded_one_hot
    
    for idx, (train_index, test_index) in enumerate(kf.split(X, y)):
        X_train, X_test = X_selected[train_index], X_selected[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
        model.fit(X_train, y_train,eval_set=[(X_test,y_test)], early_stopping_rounds=100, verbose=False)
    
        preds = model.predict(X_test)
        
        y_test_exp = np.expm1(y_test)
        preds_exp = np.expm1(preds)
        
        preds_exp = np.where(preds_exp < 0, 0, preds_exp)
        
        kf_scores[idx] = rmsle(y_test_exp, preds_exp)

    return np.mean(kf_scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)


params = study.best_trial.params
val = study.best_trial.value

params

In [ ]:

# Target encoding

# X_total = pd.concat([X, y], axis=1)

# X_encoded_target = X_total.copy()


# X_encoded_target = X_encoded_target[np.abs(X_encoded_target.revenue - X_encoded_target.revenue.mean()) <= (3 * X_encoded_target.revenue.std())]

# all_means = {}

# target_encode_features = ['mall_name', 'chain_name']

# for column in target_encode_features:
   
#    column_means = X_encoded_target.groupby(column)[column,'revenue'].mean().to_dict()['revenue']
   
#    X_encoded_target[column] = X_encoded_target.groupby(column)['revenue'].transform('mean')
#    X_encoded_target[column] = X_encoded_target[column].astype('float64')
   
#    all_means[column] = column_means

# y_target = X_encoded_target.revenue
    
# X_encoded_target.drop(columns=['revenue'], inplace=True)



# categorical_features = categorical_features.drop(['mall_name', 'chain_name'])

# full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features )], remainder='passthrough')
# one_hot_encoder = full_pipeline.fit(X_encoded_target)
# X_encoded_target_one_hot = one_hot_encoder.transform(X_encoded_target)


# X_test_encoded_target = X_test.copy()

# for column in categorical_features:
#     X_test_encoded_target[column] = X_test_encoded_target[column].map(all_means[column])
#     # convert to float
#     X_test_encoded_target[column] = X_test_encoded_target[column].astype(float)